In [ ]:
import json, os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Business Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/3rd project_recommender system/data preparation/merged_df.csv')
df.shape

(185289, 19)

In [ ]:
df = df.sample(n=50000, random_state=42)
df.head()

,business_id,name,city,state,stars_x,review_count_x,categories,user_id,text,stars_y,useful_x,funny_x,cool_x,review_count_y,elite_year_count,activeness,useful_y,funny_y,cool_y
155376,cGX-1IUwXOjkUqZbkKYcjw,Fogo de Chao,Philadelphia,PA,4.0,1426,"Seafood, Steakhouses, Brazilian, Restaurants",fJ3iKa2YmdNMOOy4L_R9kQ,i visited fogo de chao during restaurant week ...,5,0,0,0,22,0,0.0,14,8,9
14336,Ucl9Vo5lwrUmYbV8Dv8X5g,O'Briens Irish Pub,Tampa,FL,4.0,108,"Bars, Pubs, Nightlife, Arts & Entertainment, I...",SZUMrdlgShESvpC2fTgWhg,if you ever want to go where everyone knows yo...,5,0,0,0,79,2,26.0,64,20,8
117735,V7TaLZ5EB94hqRIZ1_xjGQ,Kawa Japanese Cuisine,Springfield,PA,4.5,145,"Sushi Bars, Restaurants, Ramen, Steakhouses, J...",u4ML5OJou_ShyAEfXA3-YQ,"great service, atmosphere and exceptional sush...",5,6,1,3,12,0,0.0,10,1,5
69016,cXSyVvOr9YRN9diDkaWs0Q,Honey's Sit-N-Eat,Philadelphia,PA,4.0,1396,"Southern, Restaurants, American (Traditional)",lgUKLRsqvhu4DbM9syzq_Q,ok i get it. i understand why people wait ridi...,4,5,3,2,388,7,302.0,1309,510,818
36614,cGa8ySpfmtfQKPkHJKHnUw,Michael's Family Restaurant,Montgomeryville,PA,3.0,81,"Restaurants, Breakfast & Brunch, Diners, Sandw...",GQJn9zM03Yt3ynn2VvKltw,i've been to michael's a couple of times now. ...,5,3,0,1,117,0,0.0,159,24,65


In [ ]:
df.shape 

(50000, 19)

## Data Preprocessing

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
import string
import re, os

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stemmer = SnowballStemmer('english')

def preprocess(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords, punctuation marks, and non-alphanumeric characters, and stem the remaining words
    tokens = [stemmer.stem(re.sub(r'[^a-zA-Z]', '', token.lower())) for token in tokens if not token.lower() in nltk.corpus.stopwords.words('english') and not token.lower() in string.punctuation]

    return tokens

# Tokenize and stem the reviews
df['review_tokens'] = df['text'].apply(preprocess)

# # Define a translation table to remove punctuation marks
# translator = str.maketrans('', '', string.punctuation)

# # Tokenize the reviews and remove punctuation marks
# df['review_tokens'] = df['text'].apply(lambda x: word_tokenize(x.translate(translator)))

# # Remove stop words from the tokenized reviews
# df['review_tokens'] = df['review_tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# # Perform stemming on the remaining words
# df['review_tokens'] = df['review_tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

# Create a vocabulary of unique words
vocabulary = set()
for tokens in df['review_tokens']:
    vocabulary.update(tokens)
vocabulary = sorted(list(vocabulary))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


This code uses the word_tokenize() function from the nltk.tokenize module to tokenize the reviews, the stopwords corpus from the nltk.corpus module to define a list of stop words, and the PorterStemmer and WordNetLemmatizer classes from the nltk.stem module to perform stemming or lemmatization on the remaining words. It then creates a vocabulary of unique words by iterating over the tokenized reviews and adding all unique words to a set, which is then converted to a sorted list. The result is a new column called "review_tokens" that contains a list of preprocessed tokens for each review, and a list called "vocabulary" that contains all unique words in the reviews.

In [ ]:
df[['text','review_tokens']]

,text,review_tokens
155376,i visited fogo de chao during restaurant week ...,"[visit, fogo, de, chao, restaur, week, group, ..."
14336,if you ever want to go where everyone knows yo...,"[ever, want, go, everyon, know, name, , go, ob..."
117735,"great service, atmosphere and exceptional sush...","[great, servic, atmospher, except, sushi, ever..."
69016,ok i get it. i understand why people wait ridi...,"[ok, get, understand, peopl, wait, ridicul, lo..."
36614,i've been to michael's a couple of times now. ...,"[ve, michael, s, coupl, time, first, s, import..."
...,...,...
44090,"for me, rostizado is all about the rotisserie....","[rostizado, rotisseri, love, roast, chicken, b..."
103594,we were staying at the royal sonesta for last ...,"[stay, royal, sonesta, last, sever, day, go, r..."
933,wow. i have been looking for a local indian pl...,"[wow, look, local, indian, place, nirvana, exc..."
160629,loved this place! my wife and i were in town f...,"[love, place, wife, town, weekend, happen, rit..."


In [ ]:
# import spacy
# import string
# from spacy.lang.en.stop_words import STOP_WORDS

# # Load the English model
# nlp = spacy.load('en_core_web_sm')

# # Define a set of punctuation marks and stop words
# punctuations = string.punctuation
# stop_words = set(STOP_WORDS)

# # Define a function to tokenize and lemmatize text
# import re

# def preprocess(text):
#     # Create a Doc object from the text
#     doc = nlp(text)

#     # Remove stop words and punctuation marks, and lemmatize the remaining words
#     tokens = [re.sub(r'[^a-zA-Z0-9]', '', token.lemma_.lower()) for token in doc if not token.is_stop and not token.is_punct]

#     return tokens

# # Tokenize and lemmatize the reviews
# df['review_tokens'] = df['text'].apply(preprocess)

# # Create a vocabulary of unique words
# vocabulary = set()
# for tokens in df['review_tokens']:
#     vocabulary.update(tokens)
# vocabulary = sorted(list(vocabulary))

I'm using SpaCy's en_core_web_sm model to tokenize and lemmatize the text. I define a function preprocess that takes a text as input, creates a Doc object **using SpaCy, removes stop words and punctuation marks, and lemmatizes the remaining words**. I then apply this function to the text column of merged_df using the apply method. Finally, I create a vocabulary of unique words from the tokenized and lemmatized reviews using a set.

--> can't run this cell due to time consuming

### converting reviews into numerical representations

Count-Based Representation

> TF-IDF Matrix



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the preprocessed text into a single string
df['review_text'] = df['review_tokens'].apply(lambda x: ' '.join(x))

# Define the vectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df=0.75, min_df=5, max_features=5000)

# Fit and transform the reviews
tfidf_matrix = tfidf.fit_transform(df['review_text'])

# Create DTM
dtm_tfidf = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names_out())

# Print the shape of the TF-IDF matrix
print(tfidf_matrix.shape)
display(dtm_tfidf)

(50000, 5000)


,abl,abl get,absolut,absolut amaz,absolut delici,absolut favorit,absolut love,abund,ac,acai,...,yolk,york,young,younger,yum,yum yum,yummi,zero,zone,zucchini
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.060096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I've added some additional parameters to the TfidfVectorizer:

- ngram_range=(1,2): This specifies that the vectorizer should consider both unigrams and bigrams when creating features.
- max_df=0.75: This specifies that words should be excluded from the vocabulary if they appear in more than 75% of the documents.
- min_df=5: This specifies that words should be excluded from the vocabulary if they appear in fewer than 5 documents.
- max_features=5000: This specifies that the vectorizer should consider at most 5000 features (i.e., the 5000 most frequent words in the corpus).

how to split the data into training and test sets, train a recommender system using TF-IDF, generate recommendations for each user in the test set, and calculate the MAP score

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Join the preprocessed text into a single string
df['review_text'] = df['review_tokens'].apply(lambda x: ' '.join(x))

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the vectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df=0.75, min_df=5, max_features=5000)

# Fit and transform the reviews
tfidf_matrix_train = tfidf.fit_transform(train_df['review_text'])

# Initialize cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)

# Define function to get top recommendations for each user
def recommendations(user_id, cosine_sim_matrix, df, top_n=5):
    # Get index of user_id in df
    user_index = df[df['user_id'] == user_id].index[0]

    # Get cosine similarity scores for user_index
    sim_scores = list(enumerate(cosine_sim_matrix[user_index]))

    # Sort the list of sim_scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get indices of top_n similar users
    top_similar_users = [i[0] for i in sim_scores[1:top_n+1]]

    # Get the restaurant recommendations for the top_n similar users
    recommended_restaurants = df.iloc[top_similar_users][['name', 'categories', 'stars_x']]

    return recommended_restaurants

user_id = 'fJ3iKa2YmdNMOOy4L_R9kQ'
top_n = 5
recommended_restaurants = recommendations(user_id, cosine_sim, df, top_n)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.75, min_df=5, max_features=5000)

# Fit and transform the training data
tfidf_matrix_train = tfidf.fit_transform(train_df['text'])

# Initialize cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)

# Get a list of restaurants reviewed by a given user
def get_user_reviews(user_id, df):
    return df[df['user_id'] == user_id]['business_id'].tolist()

# Find similar restaurants for a given restaurant
def get_similar_restaurants(restaurant_id, tfidf_matrix, df):
    restaurant_idx = df[df['business_id'] == restaurant_id].index[0]
    similarity_scores = cosine_similarity(tfidf_matrix[restaurant_idx], tfidf_matrix)
    similar_restaurants = list(enumerate(similarity_scores[0]))
    similar_restaurants = sorted(similar_restaurants, key=lambda x: x[1], reverse=True)
    return similar_restaurants[1:]

# Make recommendations for a given user
def recommend_restaurants(user_id, tfidf_matrix, df, top_n=5):
    user_reviews = get_user_reviews(user_id, df)
    restaurant_scores = {}
    for restaurant_id in user_reviews:
        similar_restaurants = get_similar_restaurants(restaurant_id, tfidf_matrix, df)
        for i, (idx, score) in enumerate(similar_restaurants):
            if idx in restaurant_scores:
                restaurant_scores[idx] += score * (0.95 ** i)
            else:
                restaurant_scores[idx] = score * (0.95 ** i)
    restaurant_scores = sorted(restaurant_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_restaurants = [df.iloc[idx]['business_id'] for idx, score in restaurant_scores[:top_n]]
    return recommended_restaurants

In [ ]:
# Define function to calculate Average Precision (AP) for a given user
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# Get unique user_ids in test_df
users_test = test_df['user_id'].unique()

# Initialize list to store AP scores for each user
ap_scores = []

# Generate recommendations and calculate AP for each user in test_df
for user in users_test:
    # Get recommendations for user
    recommendations = get_recommendations(user, cosine_sim, train_df)

    # Get actual restaurants rated by user in test_df
    actual = test_df[test_df['user_id'] == user]['name'].tolist()

    # Calculate AP for user
    ap = apk(actual, recommendations['name'].tolist())

    # Append AP score to ap_scores
    ap_scores.append(ap)

# Calculate MAP score
map_score = sum(ap_scores) / len(ap_scores)

print("MAP Score: ", map_score)

The main difference between the two is that CountVectorizer simply counts the number of times each word appears in a document, while TfidfVectorizer takes into account the frequency of the word in the entire corpus of documents.

TfidfVectorizer gives more weight to words that are more important or informative for a given document, while CountVectorizer treats all words equally.

Refer to https://arxiv.org/pdf/2004.13851.pdf



In [ ]:
user_id = 'your_user_id_here'
top_n = 5
recommended_restaurants = recommend_restaurants(user_id, tfidf_matrix, df, top_n)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

def calculate_apk(actual, predicted, k=10):
    """
    Calculates the Average Precision at k for a single user.
    
    Args:
    actual (list): A list of the actual restaurant IDs (as strings).
    predicted (list): A list of the predicted restaurant IDs (as strings).
    k (int): The number of recommendations to consider.
    
    Returns:
    apk (float): The Average Precision at k for the user.
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
            
    if not actual:
        return 0.0
    
    return score / min(len(actual), k)

def calculate_map(actual_dict, predicted_dict, k=10):
    """
    Calculates the Mean Average Precision for a set of recommendations.
    
    Args:
    actual_dict (dict): A dictionary mapping user IDs to lists of actual restaurant IDs.
    predicted_dict (dict): A dictionary mapping user IDs to lists of predicted restaurant IDs.
    k (int): The number of recommendations to consider.
    
    Returns:
    map (float): The Mean Average Precision for the recommendations.
    """
    apks = []
    for user_id in actual_dict.keys():
        actual = actual_dict[user_id]
        predicted = predicted_dict.get(user_id, [])
        apk = calculate_apk(actual, predicted, k=k)
        apks.append(apk)
        
    return np.mean(apks)

In [ ]:
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, MaxPooling1D, Flatten, Input, concatenate
# from keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

# # Load the data and split into train and test sets
# X = df['text']
# y = df['user_id']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Convert the text data to a TF-IDF matrix
# X_train_tfidf = tfidf.transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)

# # Define the deep learning model
# input_layer = Input(shape=(X_train_tfidf.shape[1],))
# embedding_layer = Embedding(input_dim=X_train_tfidf.shape[1], output_dim=128)(input_layer)
# conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
# max_pool_layer = MaxPooling1D(pool_size=2)(conv_layer)
# flatten_layer = Flatten()(max_pool_layer)
# dense_layer = Dense(128, activation='relu')(flatten_layer)
# output_layer = Dense(1, activation='sigmoid')(dense_layer)
# cnn_model = Model(inputs=input_layer, outputs=output_layer)

# # Compile the model
# cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# # Train the CNN model
# cnn_model.fit(X_train_tfidf, y_train, batch_size=64, epochs=5, validation_data=(X_test_tfidf, y_test))

# # Get the predictions from the CNN model
# cnn_preds_train = cnn_model.predict(X_train_tfidf)
# cnn_preds_test = cnn_model.predict(X_test_tfidf)

# # Define the traditional machine learning model
# lr_model = LogisticRegression()

# # Train the traditional machine learning model on the TF-IDF matrix
# lr_model.fit(X_train_tfidf, y_train)

# # Get the predictions from the traditional machine learning model
# lr_preds_train = lr_model.predict_proba(X_train_tfidf)[:,1]
# lr_preds_test = lr_model.predict_proba(X_test_tfidf)[:,1]

# # Concatenate the predictions from both models
# hybrid_train = np.concatenate((cnn_preds_train, lr_preds_train.reshape(-1,1)), axis=1)
# hybrid_test = np.concatenate((cnn_preds_test, lr_preds_test.reshape(-1,1)), axis=1)

# # Define the final classifier
# final_model = LogisticRegression()

# # Train the final classifier on the concatenated predictions
# final_model.fit(hybrid_train, y_train)

# # Evaluate the final model on the test set
# final_preds = final_model.predict(hybrid_test)
# accuracy = accuracy_score(y_test, final_preds)
# print('Accuracy: {}'.format(accuracy))

two classes: "liked by the user" & "not liked by the user"<br/>
-->Binary classification prob.

Distributed-Based Representation


> Embedding Matrix: Pretrained word embedding GloVe



In [ ]:
# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Dense

# # Use pre-trained GloVe embeddings
# embeddings_index = {}
# embedding_file = '/Users/username/Documents/glove.6B.100d.txt
# with open(embedding_file, 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs

# # Convert text data to sequences of word indices
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train)
# X_train_sequences = tokenizer.texts_to_sequences(X_train)
# X_test_sequences = tokenizer.texts_to_sequences(X_test)

# # Pad sequences to a fixed length
# max_sequence_length = 100
# X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
# X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# # Create embedding matrix
# word_index = tokenizer.word_index
# num_words = min(5000, len(word_index) + 1)
# embedding_matrix = np.zeros((num_words, 100))
# for word, i in word_index.items():
#     if i >= num_words:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

# # Build deep learning model
# model = Sequential()
# model.add(Embedding(input_dim=num_words, output_dim=100, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
# model.add(LSTM(64))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train model on training data
# model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluate model on test data
# score = model.evaluate(X_test_padded, y_test, batch_size=32)

# # Make restaurant recommendations
# user_preferences = 'I want a cheap Italian restaurant near downtown'
# user_preferences = ' '.join([stemmer.stem(token) for token in tokenizer(user_preferences.lower())])
# user_sequence = tokenizer.texts_to_sequences([user_preferences])
# user_padded = pad_sequences(user_sequence, maxlen=max_sequence_length)
# predicted_ratings = model.predict(user_padded)
# top_restaurants = df.loc[df['stars'].isin(np.argsort(predicted_ratings)[-10:])]

This code uses GloVe pre-trained word embeddings to create an embedding matrix, and builds a deep learning model using LSTM layers to predict restaurant ratings. It then trains the model on the training data, evaluates the model on the test data, and makes restaurant recommendations based on the user's preferences

In [ ]:
import gensim.downloader as api

# Download and load the pre-trained GloVe word embeddings
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


Gensim is an open-source Python library designed to process and analyze large-scale collections of text data. It provides implementations of several state-of-the-art algorithms for natural language processing (NLP), including topic modeling, document similarity analysis, and word embedding.

GloVe (Global Vectors for Word Representation) is a popular unsupervised algorithm for generating word embeddings, which are dense vector representations of words that capture their semantic and syntactic meaning. The GloVe algorithm is typically used to train word embeddings on large text corpora.

While GloVe itself is not directly integrated into Gensim, Gensim provides an interface for loading pre-trained GloVe embeddings and using them for downstream NLP tasks. This makes it easy to incorporate GloVe embeddings into your own NLP models built with Gensim.

## Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

## Model Selection



In [ ]:
from keras.preprocessing.text import Tokenizer

# Create a tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the list of review tokens
tokenizer.fit_on_texts(df['review_tokens'])

# Convert the list of tokenized reviews to sequences
sequences = tokenizer.texts_to_sequences(df['review_tokens'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gc

nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Define a function to convert each review to a sequence of vectors
def reviews_to_vectors_with_sentiment(reviews, tokenizer, max_length, sid, embedding_model):
    # Convert the tokenized reviews to sequences of word indices
    sequences = tokenizer.texts_to_sequences(reviews)

    # Convert the sequences of word indices to sequences of word embeddings
    vectors = []
    for seq in sequences:
        vector_seq = []
        for word_index in seq:
            word = tokenizer.index_word[word_index]
            if word in embedding_model:
                vector_seq.append(embedding_model[word])
        vectors.append(vector_seq)

    # Pad the sequences with zeros so that all reviews have the same length
    padded_vectors = pad_sequences(vectors, maxlen=max_length, padding='post')

    # Calculate sentiment scores for each review
    sentiment_scores = []
    for i, review in enumerate(reviews):
        ss = sid.polarity_scores(review)
        sentiment_scores.append(ss['compound'])
        if i % 1000 == 0:
            gc.collect()  # Call the garbage collector every 1000 reviews to free up memory
    
    sentiment_scores = np.array(sentiment_scores)
    return padded_vectors, sentiment_scores

# Convert the training and testing data to sequences of vectors
max_length = 100  # Set the maximum length of a review to 100 words
train_vectors, train_sentiment = reviews_to_vectors_with_sentiment(train_data['text'], tokenizer, max_length, sid, word_vectors)
test_vectors, test_sentiment = reviews_to_vectors_with_sentiment(test_data['text'], tokenizer, max_length, sid, word_vectors)

# Split the data into input and output arrays
X_train = train_vectors
y_train = train_sentiment
X_test = test_vectors
y_test = test_sentiment

I can use unsupervised sentiment analysis techniques such as sentiment lexicons to estimate the sentiment of your reviews. A sentiment lexicon is a collection of words and their associated sentiment scores (e.g., positive, negative, or neutral).

The sentiment_scores list returned by the reviews_to_vectors_with_sentiment function is a list of dictionaries, which cannot be directly encoded by the LabelEncoder or any other encoder.

In [ ]:
# # Build deep learning model
# model = Sequential()
# model.add(Embedding(input_dim=num_words, output_dim=100, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
# model.add(LSTM(64))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train model on training data
# model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluate model on test data
# score = model.evaluate(X_test_padded, y_test, batch_size=32)

# # Make restaurant recommendations
# user_preferences = 'I want a cheap Italian restaurant near downtown'
# user_preferences = ' '.join([stemmer.stem(token) for token in tokenizer(user_preferences.lower())])
# user_sequence = tokenizer.texts_to_sequences([user_preferences])
# user_padded = pad_sequences(user_sequence, maxlen=max_sequence_length)
# predicted_ratings = model.predict(user_padded)
# top_restaurants = df.loc[df['stars'].isin(np.argsort(predicted_ratings)[-10:])]

In [ ]:
print(train_vectors.shape)
print(y_train.shape)
print(train_vectors.dtype)
print(y_train.dtype)

## Model Training

I will train the deep neural network using the training set and validate it using the testing set.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.models import Sequential

# Define the RNN model architecture
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv.vocab), output_dim=100, weights=[tfidf], input_length=max_length))
model.add(SimpleRNN(units=128))
model.add(Dense(units=1, activation='sigmoid'))

This code defines a simple RNN model with an Embedding layer, a SimpleRNN layer, and a Dense output layer. The Embedding layer is initialized with the pre-trained word vectors, and the weights are fixed during training. The SimpleRNN layer processes the sequence of word vectors and produces a final output, which is passed through a Dense layer with a sigmoid activation function to produce a binary classification output.

I chose the RNN model with an Embedding layer, a SimpleRNN layer, and a Dense output layer because it is a popular and effective model for processing sequential data like text. The Embedding layer is used to convert the sequence of words represented by the pre-trained word embeddings into a sequence of dense vectors that can be understood by the RNN model. The SimpleRNN layer is used to process the sequence of dense vectors and capture the relationships between the words in the sequence. Finally, the Dense output layer is used to predict the sentiment of the restaurant review based on the relationships between the words captured by the RNN model.

The RNN model is a good choice for this problem because it can capture the contextual relationships between words in a sentence. It is well-suited for text classification tasks like sentiment analysis, where the order of words in a sentence is important for determining the sentiment of the sentence. Additionally, the SimpleRNN layer is a computationally efficient choice for this problem, as it is capable of capturing short-term dependencies in the input sequence.

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_vectors, train_labels, epochs=10, batch_size=64, validation_data=(test_vectors, test_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_vectors, test_labels, verbose=False)
print(f'Test accuracy: {accuracy:.3f}')

The model is then compiled with an Adam optimizer and binary cross-entropy loss function.

For binary classification problems, where the goal is to classify an input into one of two classes, binary cross-entropy is a common loss function used in neural networks. It calculates the difference between the predicted probabilities and the true class labels and updates the model weights accordingly. It is often chosen because it has desirable mathematical properties and has been shown to work well in practice.

As for the metrics, accuracy is a commonly used metric for classification problems. It measures the proportion of correctly classified examples out of all the examples. In this case, it tells us the percentage of reviews that are correctly classified as positive or negative. By using accuracy as the metric, we can easily evaluate the performance of the model and compare it to other models that use the same metric.

It is trained on the train_vectors and train_labels arrays for 10 epochs with a batch size of 64. Finally, the model is evaluated on the test_vectors and test_labels arrays, and the test accuracy is printed.

model selection not for deep learning

**Collaborative Filtering** recommends items based on similar users' preferences.<br>
**Content-based Filtering** recommends items based on the attributes of the items.<br>
**Hybrid Filtering** combines both Collaborative and Content-based filtering to improve the performance.

In [ ]:
# !pip install scikit-surprise

- I need to first create the necessary data structures to store the required information.

1. **Collaborative Filtering**: 
I will use the Surprise library to implement collaborative filtering. Surprise is a Python library for building and analyzing recommender systems that deal with explicit rating data.

2. **Content-based Filtering**:
I will use the TF-IDF matrix that I created earlier. I can compute the pairwise cosine similarity between all the items using the cosine_similarity function from scikit-learn.

3. **Hybrid Filtering**:
Once I have the necessary data structures, I can use them to build a hybrid recommendation system. The basic idea behind hybrid filtering is to combine the predictions from collaborative and content-based filtering models.

In [ ]:
# Collaborative Filtering
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Define a reader to read in the rating data
reader = Reader(rating_scale=(1, 5))

# Load the rating data into a Surprise dataset
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars_y']], reader)

# Define a collaborative filtering model using SVD
model_collab = SVD()

# Train the model on the rating data
model_collab.fit(data)

SVD can be used to perform Latent Semantic Analysis (LSA) in NLP by transforming a document-term matrix into a document-concept matrix, where the concepts are derived from the singular values and vectors of the document-term matrix. This can be useful for text classification, information retrieval, and topic modeling.




In [ ]:
# Content-based Filtering
from sklearn.metrics.pairwise import cosine_similarity

# Compute the pairwise cosine similarity between all the items
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Here, I will use a simple linear combination of the two models' predictions, where the weight given to each model is determined by a parameter alpha.

In [ ]:
# Hybrid Filtering
def hybrid_recommendations(user_id, business_id, alpha=0.5):
    """
    Get hybrid recommendations for a user and a business.
    
    Parameters:
        - user_id: ID of the user for whom to make recommendations
        - business_id: ID of the business for which to make recommendations
        - alpha: Weight given to the collaborative filtering model (default: 0.5)
    
    Returns:
        - A list of top 10 recommended business names and their corresponding predicted ratings
    """
    
    # Get the indices of the user and business in the data matrix
    user_idx = np.where(df['user_id'] == user_id)[0][0]
    business_idx = np.where(df['business_id'] == business_id)[0][0]
    
    # Compute the collaborative filtering prediction
    collab_pred = model_collab.predict(user_id, business_id).est
    
    # Compute the content-based filtering predictions
    content_preds = cosine_sim[business_idx]
    
    # Combine the predictions from both models
    hybrid_pred = (1 - alpha) * collab_pred + alpha * content_preds
    
    # Get the top 10 recommended business names and their predicted ratings
    hybrid_preds = list(enumerate(hybrid_pred))
    hybrid_preds = sorted(hybrid_preds, key=lambda x: x[1], reverse=True)
    top_business_indices = [x[0] for x in hybrid_preds[:10]]
    top_business_names = df.iloc[top_business_indices]['name'].tolist()
    top_business_ratings = [x[1] for x in hybrid_preds[:10]]
    
    return

### Challenge

Cannot set the city